## Data Cleaning

### Load Data

In [1]:
!pip install cufflinks
!pip install plotly

In [2]:
# import relevant libraries
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import datetime
import plotly.offline
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
sns.set()

In [3]:
# list of prefered dataframe names in same order as directory
customers = pd.read_csv("/project/data/raw/olist_customers_dataset.csv")
location = pd.read_csv("/project/data/raw/olist_geolocation_dataset.csv")
orders = pd.read_csv("/project/data/raw/olist_orders_dataset.csv")
order_items = pd.read_csv("/project/data/raw/olist_order_items_dataset.csv")
payments = pd.read_csv("/project/data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("/project/data/raw/olist_order_reviews_dataset.csv")
products = pd.read_csv("/project/data/raw/olist_products_dataset.csv")
sellers = pd.read_csv("/project/data/raw/olist_sellers_dataset.csv")
translation = pd.read_csv("/project/data/raw/product_category_name_translation.csv")

#### Order Reviews Dataset
This dataset includes data about the reviews made by the customers.

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

In [4]:
def replace_na_with_text(df, column, new_text):
    '''replace missing text with preferred description'''
    
    # change missing data to appropriate label
    df[column] = df[column].replace(np.nan, new_text)

In [5]:
replace_na_with_text(df=reviews, column='review_comment_title', new_text='no title')
replace_na_with_text(df=reviews, column='review_comment_message', new_text='no comment given')

In [6]:
def timestamp_to_datetime(df, columns):
    
    for col in columns:
        df[col] = pd.to_datetime(df[col])

In [7]:
columns = ['review_creation_date', 'review_answer_timestamp']
timestamp_to_datetime(reviews, columns)

In [8]:
def lowercase(df, columns):
    '''change all text to lowercase'''
    for col in columns:
        df[col] = df[col].str.lower()

In [9]:
columns = ['review_comment_title', 'review_comment_title']
lowercase(reviews, columns)

In [10]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,no title,no comment given,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,no title,no comment given,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,no title,no comment given,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,no title,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,no title,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


#### Order Dataset
This is the core dataset. From each order you might find all other information.

In [11]:
# drop rows containing NaN values, this corresponds to no delivery
orders.dropna(inplace = True)
    
# drop canceled orders (3 remaining) and reset index
index_to_drop = orders[orders.order_status == 'canceled'].index
orders.drop(index_to_drop, inplace=True)
orders.reset_index(drop=True, inplace=True)

In [12]:
columns_to_datetime = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', \
                          'order_delivered_customer_date', 'order_estimated_delivery_date']
timestamp_to_datetime(orders, columns_to_datetime)

In [13]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96455 entries, 0 to 96454
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       96455 non-null  object        
 1   customer_id                    96455 non-null  object        
 2   order_status                   96455 non-null  object        
 3   order_purchase_timestamp       96455 non-null  datetime64[ns]
 4   order_approved_at              96455 non-null  datetime64[ns]
 5   order_delivered_carrier_date   96455 non-null  datetime64[ns]
 6   order_delivered_customer_date  96455 non-null  datetime64[ns]
 7   order_estimated_delivery_date  96455 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 5.9+ MB


In [14]:
orders.describe(exclude=object).T

/tmp/ipykernel_501/3321741864.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/tmp/ipykernel_501/3321741864.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/tmp/ipykernel_501/3321741864.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.

/tmp/ipykernel_501/3321741864.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and wi

,count,unique,top,freq,first,last
order_purchase_timestamp,96455,95933,2018-08-02 12:06:07,3,2016-09-15 12:16:38,2018-08-29 15:00:37
order_approved_at,96455,88266,2018-02-27 04:31:10,9,2016-09-15 12:16:38,2018-08-29 15:10:26
order_delivered_carrier_date,96455,80094,2018-05-09 15:48:00,47,2016-10-08 10:34:01,2018-09-11 19:48:28
order_delivered_customer_date,96455,95643,2018-07-24 21:36:42,3,2016-10-11 13:46:32,2018-10-17 13:22:46
order_estimated_delivery_date,96455,445,2017-12-20 00:00:00,507,2016-10-04 00:00:00,2018-10-25 00:00:00


In [15]:
orders.describe(exclude='datetime').T

,count,unique,top,freq
order_id,96455,96455,e481f51cbdc54678b7cc49136f2d6af7,1
customer_id,96455,96455,9ef432eb6251297304e76186b10a928d,1
order_status,96455,1,delivered,96455


#### Products Dataset
This dataset includes data about the products sold by Olist.


In [16]:
translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [17]:
# drop products with no description
products.dropna(inplace = True)
products.reset_index(drop = True, inplace = True)

In [18]:
def translate_column_with_df(df_original, df_translation, column_original, column_translation): 
    '''Translate column, given df with translation'''
    
    # convert translation to dictionary to extract key, value pairs
    df_translation = df_translation.set_index(column_original)
    df_translation = df_translation.to_dict('dict')
    
    # map translations
    df_original[column_original] = df_original[column_original]\
                                               .replace(df_translation[column_translation])

In [19]:
translate_column_with_df(products, translation, 'product_category_name', 'product_category_name_english')

In [20]:
products.describe(exclude=float).T

,count,unique,top,freq
product_id,32340,32340,1e9e8ef04dbcff4541ed26657ea517e5,1
product_category_name,32340,73,bed_bath_table,3029


> Over 32,000 products within 71 categories

In [21]:
products.describe(exclude=object).T

,count,mean,std,min,25%,50%,75%,max
product_name_lenght,32340.0,48.476592,10.245699,5.0,42.0,51.0,57.0,76.0
product_description_lenght,32340.0,771.492393,635.124831,4.0,339.0,595.0,972.0,3992.0
product_photos_qty,32340.0,2.188961,1.736787,1.0,1.0,1.0,3.0,20.0
product_weight_g,32340.0,2276.956586,4279.291845,0.0,300.0,700.0,1900.0,40425.0
product_length_cm,32340.0,30.854545,16.955965,7.0,18.0,25.0,38.0,105.0
product_height_cm,32340.0,16.958813,13.636115,2.0,8.0,13.0,21.0,105.0
product_width_cm,32340.0,23.208596,12.078762,6.0,15.0,20.0,30.0,118.0


#### Sellers Dataset
This dataset includes data about the sellers that fulfilled orders made at Olist. Use it to find the seller location and to identify which seller fulfilled each product.

In [22]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [23]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


#### Customers Dataset
This dataset has information about the customer and its location. Use it to identify unique customers in the orders dataset and to find the orders delivery location.

At our system each order is assigned to a unique customerid. This means that the same customer will get different ids for different orders. The purpose of having a customerunique_id on the dataset is to allow you to identify customers that made repurchases at the store. Otherwise you would find that each order had a different customer associated with.

In [24]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [25]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


#### Order Items Dataset
This dataset includes data about the items purchased within each order.

In [26]:
timestamp_to_datetime(order_items, ['shipping_limit_date'])

In [27]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [28]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


#### Geolocation Dataset
This dataset has information Brazilian zip codes and its lat/lng coordinates. Use it to plot maps and find distances between sellers and customers.

In [29]:
location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


In [30]:
location.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [31]:
location.geolocation_zip_code_prefix.nunique()

19015

> location data seems to have many duplicates with miniscule lat and lng variations between each. Keeping only the first of each zip code will help avoid merge issues later on.

In [32]:
location.drop_duplicates(subset = ['geolocation_zip_code_prefix','geolocation_city','geolocation_state'], keep = 'first', inplace = True)

In [33]:
location.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27912 entries, 0 to 1000046
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  27912 non-null  int64  
 1   geolocation_lat              27912 non-null  float64
 2   geolocation_lng              27912 non-null  float64
 3   geolocation_city             27912 non-null  object 
 4   geolocation_state            27912 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 1.3+ MB


#### Payments Dataset
This dataset includes data about the orders payment options.

In [34]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [35]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


### Combine data into 2 dataframes

In order to analyze customer ratings and seller influences, we will combine the data into dataframes related to their respective content. For exploratory analysis we will create dataframes for users and sellers seperately, analyzing product ratings, price, revenue and sales. This will give us some insights into what aspects of user and product information that play the strongest part in predicting user habits. 

In [36]:
def merge_multiple_df_on_key(dfs, key):
    '''create dataframe with customer info including their location, order details and review'''
    
    from functools import reduce
    
    # stolen from https://stackoverflow.com/questions/23668427/pandas-three-way-joining-multiple-dataframes-on-columns)
    new_df = reduce(lambda left,right: pd.merge(left,right, how = 'inner', on=key), dfs)
    
    return new_df

In [37]:
orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [38]:
# dfs to merge with order_id column
dfs = [orders, order_items, reviews]
# merge on key
customer_purchases = merge_multiple_df_on_key(dfs, key = 'order_id')

In [39]:
def drop_duplicates_ignore_cols(df, cols_to_ignore, keep = 'first'):
    '''drop duplicates but ignore certain columns'''
    df.drop_duplicates(keep = keep, \
                       subset=df.columns.difference(cols_to_ignore), \
                       inplace=True)

In [40]:
# drop duplicate rows based on all columns except these
cols_to_ignore = ['review_id', 'review_answer_timestamp', 'review_creation_date']

# call function
drop_duplicates_ignore_cols(customer_purchases, cols_to_ignore)

In [41]:
# drop order delivery details, this will go into the seller_info df
cols_to_drop = orders.columns.difference(['order_id', 'customer_id', 'order_purchase_timestamp', \
                                          'order_delivered_customer_date', 'order_estimated_delivery_date'])
customer_purchases.drop(cols_to_drop, 1, inplace=True)

/tmp/ipykernel_501/412293989.py:4: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [42]:
# set cols to drop after merging with location and customers
cols_to_drop = ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']
    
# merge with customers and location
customer_info = pd.merge(pd.merge(customer_purchases, customers, on = 'customer_id'), location, \
                             left_on =['customer_zip_code_prefix', 'customer_city', 'customer_state'], \
                             right_on = ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state']).drop(cols_to_drop, 1).reset_index(drop=True)

/tmp/ipykernel_501/470716742.py:5: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [43]:
# need to merge product info to pull the product_category_name
customer_info = customer_info.merge(products, on = 'product_id')

In [44]:
# merge all dfs associated with seller
seller_info = pd.merge(pd.merge(pd.merge(pd.merge(order_items, \
                        sellers, on = 'seller_id'), \
                        products, on = 'product_id'), \
                        reviews, on = 'order_id'), \
                        location, how = 'inner', \
                                  left_on =['seller_zip_code_prefix', 'seller_city', 'seller_state'], \
                                  right_on = ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'])

In [45]:
# drop redundant and unnecessary columns
cols_to_drop = ['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state', 'freight_value']
seller_info.drop(cols_to_drop, 1, inplace=True)
seller_info.reset_index(drop=True, inplace=True)

/tmp/ipykernel_501/3912182199.py:3: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



In [46]:
cols_to_ignore = ['review_id', 'review_answer_timestamp', 'review_creation_date']
drop_duplicates_ignore_cols(seller_info, cols_to_ignore)

In [47]:
seller_info.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,seller_zip_code_prefix,seller_city,seller_state,product_category_name,...,product_height_cm,product_width_cm,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,geolocation_lat,geolocation_lng
0,0035c0b07126fe9c24a325216fb96064,1,ec02a5d380128f7a188e9ce8f3ddd832,8444e55c1f13cd5c179851e5ca5ebd00,2017-12-12 01:29:45,131.9,37584,congonhal,MG,cool_stuff,...,25.0,18.0,3f9a735e3351a7b5159a4d1691a0b9b2,4,no title,no comment given,2017-12-08,2017-12-09 02:28:07,-22.159376,-46.044027
1,20d07181f8cec8c3fa51212c7dd926ea,1,ec02a5d380128f7a188e9ce8f3ddd832,8444e55c1f13cd5c179851e5ca5ebd00,2017-12-29 16:39:19,131.9,37584,congonhal,MG,cool_stuff,...,25.0,18.0,2e3fc526dc85286818fed5dbe4b434f6,5,no title,no comment given,2017-12-30,2018-01-03 00:23:17,-22.159376,-46.044027
2,38a920f4e80a3be96ecbfc0cba4517d6,1,ec02a5d380128f7a188e9ce8f3ddd832,8444e55c1f13cd5c179851e5ca5ebd00,2018-04-18 13:15:23,131.9,37584,congonhal,MG,cool_stuff,...,25.0,18.0,95dbec4d9c36843e048da8c0b7b3228b,4,no title,no comment given,2018-04-17,2018-04-20 03:32:54,-22.159376,-46.044027
3,bcdbeb106ccaddd04d88f709dad0f169,1,ec02a5d380128f7a188e9ce8f3ddd832,8444e55c1f13cd5c179851e5ca5ebd00,2017-12-26 07:11:01,131.9,37584,congonhal,MG,cool_stuff,...,25.0,18.0,eeb8d7733e46c628362e1d5daabd6f82,5,no title,no comment given,2017-12-29,2017-12-29 20:30:09,-22.159376,-46.044027
4,be9352379a16b3dbe8ef3a66a85b656f,1,ec02a5d380128f7a188e9ce8f3ddd832,8444e55c1f13cd5c179851e5ca5ebd00,2017-12-21 11:28:12,131.9,37584,congonhal,MG,cool_stuff,...,25.0,18.0,4389b67e9bd50b41652066772bfc5c48,5,no title,"muito bom produto entregue antes do esperado,",2017-12-18,2017-12-19 14:23:33,-22.159376,-46.044027


In [48]:
# set dfs to have common orders between them
sellers = seller_info[seller_info.order_id.isin(customer_info.order_id)]
customers = customer_info[customer_info.order_id.isin(seller_info.order_id)]

In [49]:
sellers = sellers.dropna(subset = ['product_category_name']).reset_index(drop=True)
customers = customers.dropna(subset=['product_category_name']).reset_index(drop=True)

In [50]:
datetime_cols = ['shipping_limit_date', 'review_creation_date', 'review_answer_timestamp']
    
for col in datetime_cols:
    sellers[col] = sellers[col].astype('object')

In [51]:
datetime_cols = ['shipping_limit_date', 'review_creation_date', 'review_answer_timestamp', 'order_purchase_timestamp',\
                'order_delivered_customer_date', 'order_estimated_delivery_date']

for col in datetime_cols:
    customers[col] = customers[col].astype('object')

In [52]:
customers.to_csv(r'/project/data/int/customers.csv', index=True)
sellers.to_csv(r'/project/data/int/sellers.csv', index=True)